In [ ]:
df[~df['column1'].str.contains('[a-zA-Z]', regex=True, na=False)]
.query(“_merge == “left_only”’).drop(“_merge”,axis=1)
.sample(frac=1)

.to_string(max_colwidth=1000, index=False)

In [143]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_squared_error

def BinaryColumnCreator(df,
                        column_name,
                        new_column_name,
                        value,
                        calculation):
  
  if calculation=='>':
    df[new_column_name] = np.where(df[column_name]>value,1,0)
  elif calculation =='=':
    df[new_column_name] = np.where(df[column_name]==value,1,0)
  elif calculation =='<':
    df[new_column_name] = np.where(df[column_name]<value,1,0)
  elif calculation =='isin':
    df[new_column_name] = np.where(df[column_name].isin(value),1,0)
  elif calculation =='contains':
    df[new_column_name] = np.where(df[column_name].str.contains(value),1,0)
  elif calculation == 'dict':
    for key,value in value.items():
        try:
          df[new_column_name] = np.where((df[key]==value)&(df[new_column_name]==1),1,0)
        except:
          df[new_column_name] = np.where(df[key]==value,1,0)

In [77]:
records = []

for member_id in range(0,1000):
  deposit_val = np.random.randint(5000,100000)
  lending_val = np.random.randint(5000,100000)
  tran_val = np.random.randint(1000,10000)
  tran_vol = tran_val / 100

  for month in range(1,13):
    deposit_val = deposit_val * np.random.uniform(.15, .8)
    lending_val = lending_val * np.random.uniform(.15, .8)
    tran_val = round(tran_val * np.random.uniform(.15, .8))
    
    if tran_val>0:
      tran_vol = round(max(1,tran_val/100),0)
    else:
      tran_vol = 0

    records.append({'MEMBERNBR':member_id,
                    'MONTH':month,
                    'DEPOSIT_VAL':deposit_val,
                    'LENDING_VAL':lending_val,
                    'TRAN_VAL':tran_val,
                    'TRAN_VOL':tran_vol})

pd.set_option('display.float_format', '{:.2f}'.format)

df = pd.DataFrame(records)

BinaryColumnCreator(df,'DEPOSIT_VAL','DEPOSIT_FLAG',1000,'>')
BinaryColumnCreator(df,'LENDING_VAL','LENDING_FLAG',1000,'>')
BinaryColumnCreator(df,'TRAN_VAL','TRAN_VAL_FLAG',500,'>')
BinaryColumnCreator(df,'TRAN_VOL','TRAN_VOL_FLAG',2,'>')
BinaryColumnCreator(df,"",'ATTRITION_FLAG',{'DEPOSIT_FLAG':0,'LENDING_FLAG':0,'TRAN_VAL_FLAG':0,'TRAN_VOL_FLAG':0},'dict')


print(df['DEPOSIT_FLAG'].sum())
print(df['LENDING_FLAG'].sum())
print(df['TRAN_VAL_FLAG'].sum())
print(df['TRAN_VOL_FLAG'].sum())
print(df['ATTRITION_FLAG'].sum())

df['ATTRITION_FLAG_LAG'] = df.groupby('MEMBERNBR')['ATTRITION_FLAG'].shift(1).fillna(0)

df

4173
4086
2304
3150
6919


,MEMBERNBR,MONTH,DEPOSIT_VAL,LENDING_VAL,TRAN_VAL,TRAN_VOL,DEPOSIT_FLAG,LENDING_FLAG,TRAN_VAL_FLAG,TRAN_VOL_FLAG,ATTRITION_FLAG,ATTRITION_FLAG_LAG
0,0,1,14974.07,17470.40,460,5.00,1,1,0,1,0,0.00
1,0,2,8295.53,9494.93,362,4.00,1,1,0,1,0,0.00
2,0,3,4373.31,5254.54,114,1.00,1,1,0,0,0,0.00
3,0,4,1666.84,1238.40,82,1.00,1,1,0,0,0,0.00
4,0,5,1079.24,911.78,62,1.00,1,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
11995,999,8,258.90,124.64,1,1.00,0,0,0,0,1,1.00
11996,999,9,146.38,66.92,1,1.00,0,0,0,0,1,1.00
11997,999,10,85.14,29.20,1,1.00,0,0,0,0,1,1.00
11998,999,11,66.23,18.80,0,0.00,0,0,0,0,1,1.00


In [233]:
df[df['ATTRITION_FLAG']==1]

,MEMBERNBR,MONTH,DEPOSIT_VAL,LENDING_VAL,TRAN_VAL,TRAN_VOL,DEPOSIT_FLAG,LENDING_FLAG,TRAN_VAL_FLAG,TRAN_VOL_FLAG,ATTRITION_FLAG,ATTRITION_FLAG_LAG
5,0,6,777.89,386.02,19,1.00,0,0,0,0,1,0.00
6,0,7,460.08,74.28,15,1.00,0,0,0,0,1,1.00
7,0,8,122.96,53.70,7,1.00,0,0,0,0,1,1.00
8,0,9,60.22,30.71,4,1.00,0,0,0,0,1,1.00
9,0,10,39.66,7.43,2,1.00,0,0,0,0,1,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
11995,999,8,258.90,124.64,1,1.00,0,0,0,0,1,1.00
11996,999,9,146.38,66.92,1,1.00,0,0,0,0,1,1.00
11997,999,10,85.14,29.20,1,1.00,0,0,0,0,1,1.00
11998,999,11,66.23,18.80,0,0.00,0,0,0,0,1,1.00


In [ ]:
## Will customer Attritite Based on Previous 12M Data.

In [237]:
new_df = df[['MEMBERNBR']].drop_duplicates('MEMBERNBR')

for month in df['MONTH'].unique():
  if len(new_df)==0:
    new_df = df[df['MONTH']==month][['MEMBERNBR','DEPOSIT_VAL','LENDING_VAL','TRAN_VAL','TRAN_VOL','ATTRITION_FLAG']].copy()
    new_df = new_df.rename(columns={x:x if x=='MEMBERNBR' else f"{x}_{month}M" for x in temp_df.columns})

  else:
    temp_df = df[df['MONTH']==month][['MEMBERNBR','DEPOSIT_VAL','LENDING_VAL','TRAN_VAL','TRAN_VOL','ATTRITION_FLAG']].copy()
    temp_df = temp_df.rename(columns={x:x if x=='MEMBERNBR' else f"{x}_{month}M" for x in temp_df.columns})
    new_df = new_df.merge(temp_df,on='MEMBERNBR',how='left')

new_df = new_df.merge(df[df['MONTH']==12][['MEMBERNBR','ATTRITION_FLAG']],on='MEMBERNBR',how='left')

In [239]:
# Will customer Attrite Based on Last Month of Data.

df2 = pd.DataFrame()

for month_int in range(1,12):
    d = new_df[['MEMBERNBR',f'DEPOSIT_VAL_{month_int}M',f'LENDING_VAL_{month_int}M',f'TRAN_VAL_{month_int}M',f'TRAN_VOL_{month_int}M',f'ATTRITION_FLAG_{month_int}M',f'DEPOSIT_VAL_{month_int+1}M',f'LENDING_VAL_{month_int+1}M',f'TRAN_VAL_{month_int+1}M',f'TRAN_VOL_{month_int+1}M',f'ATTRITION_FLAG_{month_int+1}M']].rename(columns={f'DEPOSIT_VAL_{month_int}M':'DEPOSIT_VAL_1M',f'LENDING_VAL_{month_int}M':"LENDING_VAL_1M",f'TRAN_VAL_{month_int}M':'TRAN_VAL_1M',f'TRAN_VOL_{month_int}M':'TRAN_VOL_1M',f'ATTRITION_FLAG_{month_int}M':'ATTRITION_FLAG_1M',f'DEPOSIT_VAL_{month_int+1}M':'DEPOSIT_VAL_2M',f'LENDING_VAL_{month_int+1}M':"LENDING_VAL_2M",f'TRAN_VAL_{month_int+1}M':'TRAN_VAL_2M',f'TRAN_VOL_{month_int+1}M':'TRAN_VOL_2M',f'ATTRITION_FLAG_{month_int+1}M':'ATTRITION_FLAG_2M'})
    df2 = pd.concat([df2,d])

df3 = df2[df2['ATTRITION_FLAG_1M']==0]


In [269]:
def MLManualPipeline(df,
                     X_Cols,
                     y_Col,
                     scaler='MinMaxScaler',
                     model_list=['Linear Regression'],
                     test_size=.3,
                     random_state=42):

    if len(X_Cols) == 0:
        X = np.array(df.drop(y_Col,axis=1).copy())
    else:
        X = np.array(df[X_Cols])
    
    y = df[y_Col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if scaler =='MinMaxScaler':
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)

    if len(model_list)==0:
        return X_train,X_test,y_train,y_test
    
    else:
        for model in model_list:
            if model == 'Linear Regression':
                lr = LinearRegression()
                lr.fit(X_train, y_train)
                y_pred_lr = lr.predict(X_test)
                
            elif model =='Logistic Regression':
                logreg=LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)
                print(f"Logisitic Regression Model:\n{confusion_matrix(y_test, y_pred)}\n{classification_report(y_test, y_pred)})")

            elif model =='Random Forest':
                
                ############################################ ESTIMATORS
                rf = RandomForestClassifier(random_state=random_state, n_estimators=25)
                rf.fit(X_train, y_train)
                y_pred_rf = rf.predict(X_test)
                print(f"Random Forest with 25 Nodes?>?>?>:\n{confusion_matrix(y_test, y_pred_rf)}\n{classification_report(y_test, y_pred_rf)})")

MLManualPipeline(df=df3.drop(['MEMBERNBR','ATTRITION_FLAG_1M'],axis=1),
                 X_Cols="",
                 y_Col='ATTRITION_FLAG_2M',
                model_list=['Logistic Regression','Random Forest'])



Logisitic Regression Model:
[[1225    6]
 [ 181  113]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1231
           1       0.95      0.38      0.55       294

    accuracy                           0.88      1525
   macro avg       0.91      0.69      0.74      1525
weighted avg       0.89      0.88      0.86      1525
)
Random Forest with 25 Nodes?>?>?>:
[[1231    0]
 [  20  274]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1231
           1       1.00      0.93      0.96       294

    accuracy                           0.99      1525
   macro avg       0.99      0.97      0.98      1525
weighted avg       0.99      0.99      0.99      1525
)


/Users/derekdewald/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/derekdewald/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/derekdewald/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/derekdewald/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` i

In [ ]:
def ScaleColumn(array,scale_method='MinMaxScaler'):
    
    if scale_method == 'MinMaxScaler':
        scaler = MinMaxScaler()
    return scaler.fit_transform(array)
   

In [ ]:
# Preprocessing: Pivot data into 3D array (members, time steps, features)
features = ['DEPOSIT_VAL', 'LENDING_VAL', 'TRAN_VAL', 'TRAN_VOL']
target = 'ATTRITION_FLAG'

# Group by Member_ID and pivot into 3D array
grouped = df.groupby('MEMBERNBR')
X = np.array([grouped[features].get_group(i).values for i in df['MEMBERNBR'].unique()])
y = np.array([grouped[target].get_group(i).values[-1] for i in df['MEMBERNBR'].unique()])

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Predict and display results
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Predictions:", y_pred_classes.flatten())
print("Actual:", y_test)

# Plot accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.tight_layout()
plt.show()